In [ ]:
import cx_Oracle
import requests
import datetime
import pandas as pd
from decimal import Decimal

In [ ]:
CONNECTION_STRING = ""
orcl = cx_Oracle.connect(CONNECTION_STRING)
orcl.autocommit = True
cursor = orcl.cursor()

In [ ]:
evds_key = "" # This must be populated before run the notebook

# Following two entries must be unchanged!
EVDS_TIME_FORMAT_STRING = "%d-%m-%Y"

EVDS_DATE_NAME = "Tarih"

# DataFrame to collect the data
evds_data = pd.DataFrame()

In [ ]:
today = datetime.datetime.now()
last_data_date = datetime.datetime(2021,8,3) ## Supposed to be get from database
start_date = last_data_date + datetime.timedelta(days=1)


In [ ]:
series_dict = {
    "PERSONAL":"TP.KTF10",
    "VEHICLE":"TP.KTF11",
    "HOUSING":"TP.KTF12"
}

In [ ]:
url_base = "https://evds2.tcmb.gov.tr/service/evds/series="
url_series_list = "-".join(list(series_dict.values()))
url_start_date = start_date.strftime(EVDS_TIME_FORMAT_STRING)
url_end_date = today.strftime(EVDS_TIME_FORMAT_STRING)
url_type = "json"
url_key = evds_key

In [ ]:
def create_evds_url(series_list, start_date, end_date):
    url = url_base + series_list + "&startDate=" + start_date + \
        "&endDate=" + end_date + "&type=json" + "&key=" + evds_key
    return url


In [ ]:
url = create_evds_url(
    url_series_list,
    url_start_date,
    url_end_date)

result = requests.get(url).json()["items"]

for x in result:
    for j in series_dict:
        key = series_dict[j].replace(".","_")
        row_dict = {
            "DATA_DATE": datetime.datetime.strptime(x[evds_date_name], evds_time_format_string),
            "VALUE": (Decimal(x[key]) / 100),
            "DATA_NAME": "MARKET_RATE_" + j,
            "DATA_TYPE": "percentage"
        }
        evds_data = evds_data.append(row_dict, ignore_index=True)

In [ ]:
sql = "insert into waited_average_interest_rate value (:1,:2,:3,:4)"

for x in evds_data.values.tolist():
    cursor.execute(sql,x)
    print(x)

